### <span style="color:red">IMPORTANT: Only modify cells which have the following comment:</span>
```python
# Modify this cell
```
##### <span style="color:red">Do not add any new cells when you submit the homework</span>

# Setting Up Notebook

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.AssertionError: assertion failed: Expected hostname
	at scala.Predef$.assert(Predef.scala:170)
	at org.apache.spark.util.Utils$.checkHost(Utils.scala:931)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:31)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:121)
	at org.apache.spark.scheduler.local.LocalEndpoint.<init>(LocalSchedulerBackend.scala:59)
	at org.apache.spark.scheduler.local.LocalSchedulerBackend.start(LocalSchedulerBackend.scala:126)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:156)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:509)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.refle

In [3]:
import numpy as np
import math
from numpy import linalg as LA

# Exercise:
The function **computeCov** computes the covariance matrix using RDDs. The code allows undefined entries and calculates the covariance without bias. 

Your homework is to complete the missing parts in **computeCov** (Marked with `...`) so that it calculates the covariance correctly.

    Note: The functions and libraries in the cell below will be useful to you

In [4]:
def outerProduct(X):
    """Computer outer product and indicate which locations in matrix are undefined"""
    O=np.outer(X,X)
    N=1-np.isnan(O)
    return (O,N)

def sumWithNan(M1,M2):
    """Add two pairs of (matrix,count)"""
    (X1,N1)=M1
    (X2,N2)=M2
    N=N1+N2
    X=np.nansum(np.dstack((X1,X2)),axis=2)
    return (X,N)

In [67]:
# Modify this cell

def computeCov(RDDin):
    # input: RDDin is an RDD of np arrays, all of the same length
    
    # we insert 1 at the beginning of each vector so the calculation also yields the mean vector
    RDD=RDDin.map(lambda v:np.array(np.insert(v,0,1),dtype=np.float64)) 
    
    # separating map and reduce does not matter, since Spark uses lazy execution.
    OuterRDD=RDD.map(lambda x: outerProduct(x))    #<-- do mapping here
    (S,N)=OuterRDD.reduce(lambda x,y: sumWithNan(x,y))  #<-- do reducing here
    
    E=S[0,1:]
    NE=np.float64(N[0,1:])
    print 'shape of E=',E.shape,'shape of NE=',NE.shape
    Mean=E/NE
    O=S[1:,1:]
    NO=np.float64(N[1:,1:])
    
    Cov=  O/NO-np.outer(Mean,Mean) # This is the covariance matrix
    
    # Output also the diagnal which is the variance for each day
    Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
    return {'E':E,'NE':NE,'O':O,'NO':NO,'Cov':Cov,'Mean':Mean,'Var':Var}

RDDin=sc.parallelize([np.array([3,2]), np.array([7,4])])
computeCov(RDDin)

shape of E= (2,) shape of NE= (2,)


{'Cov': array([[ 4.,  2.],
        [ 2.,  1.]]),
 'E': array([ 10.,   6.]),
 'Mean': array([ 5.,  3.]),
 'NE': array([ 2.,  2.]),
 'NO': array([[ 2.,  2.],
        [ 2.,  2.]]),
 'O': array([[ 58.,  34.],
        [ 34.,  20.]]),
 'Var': array([ 4.,  1.])}

In [68]:
import Tester.SmallPCA as pca
pca.exercise(computeCov, sc)

Checking data_list of length 3 with length 10 vectors each having 2 np.NaN values
shape of E= (10,) shape of NE= (10,)

Checking data_list of length 100 with length 10 vectors each having 4 np.NaN values
shape of E= (10,) shape of NE= (10,)

Great Job!


In [69]:

sc.stop()







